<a href="https://colab.research.google.com/github/cccaaannn/useful_functions/blob/master/useful_functions/01-external_modules/keras/keras_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

imports

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# keras and tf
import tensorflow as tf
import keras

# models
from keras.models import Sequential
from keras.models import Model

# backend
from keras import optimizers, metrics, models
import keras.backend as K

# layers
from keras.layers import Input, Flatten, Dense, Dropout, Activation
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D, BatchNormalization

# optimizers
from keras.optimizers import SGD

# training
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler

# save
import h5py

# keras aplications
from keras.applications import DenseNet201, DenseNet169, InceptionResNetV2, ResNet152V2, InceptionV3, DenseNet121, Xception, MobileNet


# other libs
import math
import pickle
import os

select tf 1.x

In [ ]:
%tensorflow_version 1.x

data generator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

image_size = 224
batch_size = 32


train_datagen = ImageDataGenerator(
        # rotation -40 40
        rotation_range=40,

        # shift images and fill pixels with nearest
        width_shift_range=0.1,
        height_shift_range=0.1,
        fill_mode='nearest',

        horizontal_flip=True,
        zoom_range=0.2,

        shear_range=0.2,

        brightness_range=[0.1, 0.5],
        channel_shift_range=0.5,

        # normalization 
        featurewise_center=True,
        featurewise_std_normalization=True,
        # validation_split=0.2,
        )

valid_datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)


# train = train_datagen.flow_from_directory("/content/drive/My Drive/Data/train", shuffle=True, class_mode='categorical', target_size=(image_size,image_size), batch_size=batch_size)
# valid = valid_datagen.flow_from_directory("/content/drive/My Drive/Data/test", shuffle=True, class_mode='categorical', target_size=(image_size,image_size), batch_size=batch_size)

train = valid_datagen.flow_from_directory("/content/drive/My Drive/clean_data/train", target_size=(image_size,image_size), batch_size=batch_size)
valid = valid_datagen.flow_from_directory("/content/drive/My Drive/clean_data/test", target_size=(image_size,image_size), batch_size=batch_size)

convert model to sequential and add last layer

In [ ]:
model = Sequential()
for layer in base_model.layers:
    model.add(layer)

model.add(GlobalAveragePooling2D())
model.add(Dense(100, activation='softmax'))

optimizer

In [ ]:
opt = SGD(lr=0.001, momentum=0.9)
opt = Adam(learning_rate=0.001)

top k accuracy

In [ ]:
import functools
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

top5_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=5)
top5_acc.__name__ = 'top5_acc'

top10_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=10)
top10_acc.__name__ = 'top10_acc'


opt = SGD(lr=0.001, momentum=0.9)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy', top3_acc, top5_acc, top10_acc])


# 'top_k_categorical_accuracy' = top5
# model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy', 'top_k_categorical_accuracy', top3_acc])

model checkpointer and learning schedular

In [ ]:
model_path = "/content/drive/My Drive/model"

checkpointer = ModelCheckpoint(filepath= model_path+'-{epoch:02d}-{val_loss:0.5f}-{val_accuracy:0.5f}.hdf5', verbose=1, monitor='val_loss', save_best_only=True)

def schedule(epoch):
    if epoch < 15:
        return 0.001
    elif epoch < 20:
        return 0.0001
    else:
        return 0.00001

lr_scheduler = LearningRateScheduler(schedule)

history = model.fit_generator(train, epochs=100, verbose=1, validation_data=valid, callbacks=[lr_scheduler, checkpointer])

plot model

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')